In [1]:
%run base.ipynb

# EPIC 1 - User Story DF-24
## Erstellung eines Notebooks und aufrufen der Daten per Dataland-API

Als QA-Lab-Team möchten wir ein Notebook erstellen, mit dem wir eine Anfrage an die Dataland-API senden, sodass wir anhand der Berichts-ID den Unternehmensbericht erhalten.

Beschreibung:<br>
Nutzen von Dataland Endpoint /data/nuclear-and-gas/companies/{companyId} im Nuclear & Gas Controller. Die Funktionalität wird außerhalb des Notebooks implementiert.

Akzeptanzkriterien:<br>
    - Notebook ist auf jedem Laptop ausführbar<br>
    - Anfrage an Dataland gibt Daten zurück

## 1.Schritt: Dataset mit Hilfe der Company_ID über die Dataland_API aufrufen
Mithilfe der Company_ID wird die Dataland_API aufgerufen. Nachdem Eintragen aus welcher Periode das Dataset sein soll, ist es möglich den Wert 1 zu erlangen.

In [2]:
import dataland_qa_lab.dataland.get_data as qa

company_id = "9f6f6cea-7316-4101-aba9-170023c811e8"
year = "2020"
wert1 = qa._get_data_set_by_year_(company_id=company_id, year=year)
print(wert1)


No


## 2.Schritt: Laden aller Daten
Hier werden die Daten von Dataland geladen und alle 6 Werte gesondert gespeichert.

In [3]:
from dataland_qa_lab.utils import config

conf = config.get_config()
dataland_client = conf.dataland_client

dataset = dataland_client.eu_taxonomy_nuclear_and_gas_api.get_all_company_nuclear_and_gas_data(company_id=company_id)

# Eintragen aus welcher Periode man das Dataset haben will -> In diesem Fall 2024
data_id = "test"
for t in range(len(dataset)):
    if (dataset[t].meta_info.reporting_period == year):
        data_id = dataset[t].meta_info.data_id
        break

data = dataland_client.eu_taxonomy_nuclear_and_gas_api.get_company_associated_nuclear_and_gas_data(data_id=data_id)

wert1 = data.data.general.general.nuclear_energy_related_activities_section426
wert2 = data.data.general.general.nuclear_energy_related_activities_section427
wert3 = data.data.general.general.nuclear_energy_related_activities_section428
wert4 = data.data.general.general.fossil_gas_related_activities_section429
wert5 = data.data.general.general.fossil_gas_related_activities_section430
wert6 = data.data.general.general.fossil_gas_related_activities_section431
print(data)
print(wert1)
print(wert2)
print(wert3)
print(wert4)
print(wert5)
print(wert6)

company_id='9f6f6cea-7316-4101-aba9-170023c811e8' reporting_period='2020' data=NuclearAndGasData(general=NuclearAndGasGeneral(general=NuclearAndGasGeneralGeneral(referenced_reports={'SustainabilityReport': CompanyReport(file_reference='50a36c418baffd520bb92d84664f06f9732a21f4e2e5ecee6d9136f16e7e0b63', file_name=None, publication_date=datetime.date(2023, 8, 2))}, nuclear_energy_related_activities_section426=ExtendedDataPointYesNo(value=<YesNo.NO: 'No'>, quality=<QualityOptions.REPORTED: 'Reported'>, comment='calculate wireless firewall', data_source=ExtendedDocumentReference(page='391', tag_name='synergies', file_name='SustainabilityReport', file_reference='50a36c418baffd520bb92d84664f06f9732a21f4e2e5ecee6d9136f16e7e0b63')), nuclear_energy_related_activities_section427=ExtendedDataPointYesNo(value=None, quality=<QualityOptions.INCOMPLETE: 'Incomplete'>, comment=None, data_source=ExtendedDocumentReference(page='1164', tag_name='methodologies', file_name='SustainabilityReport', file_refer

## 3.Schritt: Datenquelle laden
In diesem Fall wird die Dateireferenz von wert1 genutzt um das dazugehörige Dokument zu bekommen.

In [4]:
# datenQuelle = wert1.data_source.file_reference
# print(datenQuelle)

document_bytes = dataland_client.documents_api.get_document(wert1.data_source.file_reference)

## 4.Schritt: Daten aus dem Dokument extrahieren und in Text umwandeln

In [5]:
import io

import pypdf

full_document_byte_stream = io.BytesIO(document_bytes)
full_pdf = pypdf.PdfReader(full_document_byte_stream)

partial_document_byte_stream = io.BytesIO()
partial_pdf = pypdf.PdfWriter()

partial_pdf.add_page(full_pdf.get_page(int(wert1.data_source.page) - 1))  # Correct for 0 offset
partial_pdf.write(partial_document_byte_stream)
partial_document_byte_stream.seek(0)
None

IndexError: list index out of range

In [ ]:
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, ContentFormat
from azure.core.credentials import AzureKeyCredential

docintel_cred = AzureKeyCredential(conf.azure_docintel_api_key)
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=conf.azure_docintel_endpoint, credential=docintel_cred
)

poller = document_intelligence_client.begin_analyze_document(
    "prebuilt-layout",
    analyze_request=partial_document_byte_stream,
    content_type="application/octet-stream",
    output_content_format=ContentFormat.MARKDOWN,
)
result: AnalyzeResult = poller.result()

#### Das Ergebnis wird als Markdown direkt im Notebook wiedergegeben

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result.content))

Nein
